# Developing Custom GRASS Tools - FOSS4G 2022 Workshop

Learn how to develop custom tools (aka addons or modules) for GRASS GIS in Python and, if you like, in C.

Python scripting is powerful, but what is even more powerful is turning a script into a GRASS tool with just a few tricks and tweaks we will cover in this workshop. When you develop a GRASS tool (aka module), you get a graphical user interface (GUI), command line interface, and convenience you and your users will appreciate. Such tools can be published in a community-maintained addon repository which helps not only to distribute the tool, but also to maintain the code in the long term.

We will focus on Python, but we will cover tools written in C, too, because even compiled tools in C and C++ can be in this community-maintained repository and distributed to users.

## Authors

### Vaclav Petras

Vaclav (Vashek) Petras is a research software engineer, open source developer, and open science advocate. He received his masters in Geoinformatics from the Czech Technical University and PhD in Geospatial Analytics from the North Carolina State University. Vaclav is a member of the GRASS GIS Development Team and Project Steering Committee.

### Anna Petrasova

Anna is a geospatial research software engineer with PhD in Geospatial Analytics. She develops spatio-temporal models of urbanization and pest spread across landscape. As a member of the OSGeo Foundation and the GRASS GIS Project Steering Committee, Anna advocates the use of open source software in research and education.

Thanks for providing feedback goes to: Bernardo Santos

## Related talks

* _Take-Home Messages from Adding Code Quality Measures to GRASS GIS_
* _Tips for parallelization in GRASS GIS in the context of land change modeling_
* _Using GRASS GIS in Jupyter Notebooks: An Introduction to grass.jupyter_
* _State of GRASS GIS_

## Outline

- This notebook
  * Python script structure
  * Running Python scripts
  * Running GRASS GIS
- Tools for GRASS GIS in Python
- Best practices for writing GRASS tools
- Best practices for writing GRASS tools
- Tools for GRASS GIS in C

## Workshop Software Setup

The workshop material assumes it runs in the prepared Binder environment which is running Ubuntu and GRASS GIS is already installed there.

The Binder is set up with development version of GRASS GIS 8.3, but the notebooks will work with 8.2 as well.

## Getting GRASS GIS Ready

If you are running the notebook in the prepared Binder, there is nothing to do. If you are compiling GRASS GIS yourself, start JupyterLab in a way that your compiled GRASS GIS is the first _grass_ command on the path, e.g.,:

```bash
PATH=~/grass/code/bin.x86_64-pc-linux-gnu/:$PATH jupyter lab
```

For other cases, please refer to [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

Check that GRASS GIS is running and that you get the expected version:

In [ ]:
!grass --version

## Getting Data Ready

The Binder setup already has the [full North Carolina sample dataset](https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.zip) included.

To do our test runs in an isolated environment, we will create a new mapset (aka subproject) called _foss4g_:

In [ ]:
!grass -e -c ~/grassdata/nc_basic_spm_grass7/foss4g

To start over later on, you can use a different mapset or delete this one using:

```bash
rm -r ~/grassdata/nc_basic_spm_grass7/foss4g
```

## Python Script

Let's start with a basic Python script which uses GRASS GIS. GRASS Python packages are usually not on Python path, so we will use GRASS command line interface to get the path to these packages before we import them.

The script starts a GRASS session and uses the mapset we created above. Then, it prints the current mapset name. 

In [ ]:
%%python
# Import standard Python packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.script.setup  # Needed only in 8.2 and older.

# Use GRASS session as a context manager.
with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
    print(gs.read_command("g.mapset", flags="p"))

## Running Python Scripts from Command Line

For testing a script and for integrating it in GRASS GIS, it is advantageous to see how a script is executed from command line or generally as a subprocess.

Before, we used IPython kernel cell magic `%%python` to run a cell as a separate Python script. Now, we will use `%%writefile` cell magic to create a Python file which we will execute in the following cell.

In [ ]:
%%writefile mapset_print_script.py
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.script.setup

with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
    print(gs.read_command("g.mapset", flags="p"))

Use _python_ to execute the script. It's name (or path) are provided as parameter:

In [ ]:
!python mapset_print_script.py

## Full Python Script Structure

The best practice for Python scripts is to use a _main_ function which is called from the so-called "if name equals main" block. The name of the of the _main_ function is not import, although it usually is _main_, while the syntax of the if-name-equals-main block is. Generally, all code should be in the _main_ function or called from it. This is the structure we will use from now on:

```python
def main():
    pass

if __name__ == "__main__":
    main()
```

Our script, combined with the new structure, now looks like this:

In [ ]:
%%writefile mapset_print_main.py
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.script.setup


def main():
    with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
        print(gs.read_command("g.mapset", flags="p"))


if __name__ == "__main__":
    main()

In [ ]:
!python mapset_print_main.py

## Executable Scripts and Shebang

On unix-like systems (Linux, macOS, ...), specifying the Python interpreter can be avoided when the script has execute permissions and the first line of the script called shebang specifies which interpreter to use for the given script. A minimal script then looks like this:

```python
#!/usr/bin/env python

def main():
    pass

if __name__ == '__main__':
    main()
```

The first line now caries very special meaning, but for Python it is just a comment, although some helper tools may recognize it.

Let's add shebang to our script:

In [ ]:
%%writefile mapset_print_executable.py
#!/usr/bin/env python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.script.setup


def main():
    with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
        print(gs.read_command("g.mapset", flags="p"))


if __name__ == "__main__":
    main()

Permissions are managed using _chmod_. `chmod u+x` makes a file executable for the user:

In [ ]:
!chmod u+x mapset_print_executable.py

Script can then run without specifying the Python interpreter:

In [ ]:
!./mapset_print_executable.py

Note `./` which says the script is in the current directory. The path is always mandatory in this case even if it is the current directory. Installed GRASS tools are _on path_, i.e., are where the operating system looks for executables, so for installed tools, no path needs to be specified.

The executable mechanism on Windows is different and GRASS GIS does number of steps to ensure that the scripts can be executed and right Python is used.

## Running in GRASS GIS

GRASS tools are different from Python scripts which are using GRASS GIS in the way that they are not setting up their own GRASS session. The tools are already running in a session which was previously set up by the user in some interactive or automated way, e.g., using GUI in a desktop environment or from a Python script.

The following script assumes that it runs in a GRASS session. Because we separated the concern about the GRASS session, the script is simpler: There is no need to set up path to GRASS packages and initialize GRASS session with a mapset.

In [ ]:
%%writefile mapset_print_tool.py
#!/usr/bin/env python

import subprocess
import sys

import grass.script as gs


def main():
    print(gs.read_command("g.mapset", flags="p"))


if __name__ == "__main__":
    main()

The script can then run in an interactive GRASS session (from GUI or shell) or it can be executed using the `--exec` interface:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec python ./mapset_print_tool.py

Let's make the script executable:

In [ ]:
!chmod u+x mapset_print_tool.py

For the executable script, we can leave out `python`:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./mapset_print_tool.py

## Command Line Parameters

Scripts like all other programs, can take command line parameters. This is a crucial feature for developing general scripts and GRASS tools.

Here is a simple script which prints parameters received on the command line:

In [ ]:
%%writefile command_line_print.py
#!/usr/bin/env python

import sys


def main():
    print(f"Parameters are: {sys.argv}")


if __name__ == "__main__":
    main()

Make the script executable:

In [ ]:
!chmod u+x command_line_print.py

Try different combinations of parameters:

In [ ]:
!./command_line_print.py abc xyz 1 2 3 "dd ee ff" '44 55 66'

The script works just the same with `grass ... --exec` where parameters go after the script:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./command_line_print.py abc "dd ee ff"